In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression #DEBUG

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

In [2]:
DATA_PATH = '../data/'

df = pd.read_csv(DATA_PATH + 'processed/train_folds.csv')
df.head()

,n,p,f,x,y,z,a1,a2,a3,a4,...,g1g20.95,l1l20.05,l1l20.25,l1l20.75,l1l20.95,l3l40.05,l3l40.25,l3l40.75,l3l40.95,kfold
0,709,0.7,0.2,16,12,7,3.8,0.24,2.3,0.28,...,1.008427,-2.320280,-0.864735,1.158735,2.614280,-2.542766,-0.990184,1.168184,2.720766,0.0
1,921,0.6,0.3,19,17,10,3.0,0.12,2.4,0.19,...,2.052824,-2.445766,-0.893184,1.265184,2.817766,-2.884737,-1.138082,1.290082,3.036737,4.0
2,177,0.8,0.4,14,12,5,3.2,0.17,1.8,0.18,...,2.338795,-1.103398,-0.424143,0.520143,1.199398,-0.913912,-0.331694,0.477694,1.059912,3.0
3,415,0.7,0.5,24,11,2,1.3,0.17,1.5,0.18,...,1.872339,-1.164883,-0.388592,0.690592,1.466883,-1.787339,-0.719939,0.763939,1.831339,0.0
4,802,0.8,0.4,21,10,3,4.4,0.15,2.6,0.13,...,3.169222,-3.002222,-1.158531,1.404531,3.248222,-1.763339,-0.695939,0.787939,1.855339,4.0


In [3]:
def treinar_fold(fold, regressor):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    x_train = df_train.drop(['kfold', 'cent_price_cor', 'cent_trans_cor'], axis=1)
    y_train = df_train[['cent_price_cor', 'cent_trans_cor']]
    x_valid = df_valid.drop(['kfold', 'cent_price_cor', 'cent_trans_cor'], axis=1)
    y_valid = df_valid[['cent_price_cor', 'cent_trans_cor']]
    
    regressor.fit(x_train, y_train, verbose=False)
    y_pred = regressor.predict(x_valid)
    
    mse = mean_squared_error(y_valid, y_pred)
    print(f"Fold: {fold}, MSE = {mse}")
    
    df_valid.loc[:, 'catboost_pred_price'] = y_pred[:, 0]
    df_valid.loc[:, 'catboost_pred_trans'] = y_pred[:, 1]
    
    return df_valid, mse

In [4]:
reg = MultiOutputRegressor(CatBoostRegressor())

dfs = []
mean_mse = 0;
for i in range(5):
    fold_df, fold_mse = treinar_fold(i, reg)
    mean_mse += fold_mse
    dfs.append(fold_df)
mean_mse /= 5
    
fin_valid_df = pd.concat(dfs)
print(f'MSE Media:{mean_mse}')
print(fin_valid_df.shape)
fin_valid_df.to_csv('model_preds/catboost.csv', index=False)

Fold: 0, MSE = 0.013398114733473493
Fold: 1, MSE = 0.014367037091852948
Fold: 2, MSE = 0.014661527265402988
Fold: 3, MSE = 0.01374154873231635
Fold: 4, MSE = 0.014195386932959082
MSE Media:0.014072722951200971
(11940, 69)


In [5]:
pd.read_csv('model_preds/catboost.csv')

,n,p,f,x,y,z,a1,a2,a3,a4,...,l1l20.25,l1l20.75,l1l20.95,l3l40.05,l3l40.25,l3l40.75,l3l40.95,kfold,catboost_pred_price,catboost_pred_trans
0,709,0.7,0.2,16,12,7,3.8,0.24,2.3,0.28,...,-0.864735,1.158735,2.614280,-2.542766,-0.990184,1.168184,2.720766,0.0,-0.176531,0.364038
1,415,0.7,0.5,24,11,2,1.3,0.17,1.5,0.18,...,-0.388592,0.690592,1.466883,-1.787339,-0.719939,0.763939,1.831339,0.0,-0.169957,0.352298
2,231,0.7,0.3,21,12,8,4.7,0.28,2.0,0.23,...,-0.345143,0.599143,1.278398,-1.250883,-0.474592,0.604592,1.380883,0.0,-0.199793,0.366071
3,286,0.8,0.3,15,14,2,2.8,0.18,3.2,0.14,...,-0.362143,0.582143,1.261398,-2.884737,-1.138082,1.290082,3.036737,0.0,-0.239445,0.370899
4,788,0.7,0.5,27,12,9,2.0,0.25,2.9,0.14,...,-0.422592,0.656592,1.432883,-2.093310,-0.831837,0.921837,2.183310,0.0,-0.231418,0.349692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11935,660,0.9,0.3,24,18,4,4.1,0.20,2.4,0.20,...,-0.543490,0.805490,1.775854,-1.462368,-0.589041,0.625041,1.498368,4.0,-0.204709,0.351965
11936,592,0.8,0.3,21,18,3,2.0,0.19,1.5,0.22,...,-0.165245,0.509245,0.994427,-3.098222,-1.254531,1.308531,3.152222,4.0,-0.160040,0.365911
11937,511,0.5,0.5,28,28,3,3.9,0.25,1.6,0.23,...,-0.830286,1.058286,2.416795,-1.632854,-0.662490,0.686490,1.656854,4.0,-0.226726,0.368060
11938,255,0.6,0.5,20,28,2,4.4,0.28,1.2,0.24,...,-0.662490,0.686490,1.656854,-1.120398,-0.441143,0.503143,1.182398,4.0,-0.208116,0.354172


In [6]:
test_df = pd.read_csv(DATA_PATH + 'processed/test_input.csv')
test_df.head()

,n,p,f,x,y,z,a1,a2,a3,a4,...,g1g20.75,g1g20.95,l1l20.05,l1l20.25,l1l20.75,l1l20.95,l3l40.05,l3l40.25,l3l40.75,l3l40.95
0,558,0.5,0.2,28,21,2,1.0,0.29,1.3,0.21,...,1.412082,3.158737,-3.140707,-1.199980,1.497980,3.438707,-2.718251,-1.068633,1.224633,2.874251
1,910,0.9,0.3,28,19,9,2.2,0.14,1.6,0.14,...,0.846490,1.816854,-1.229883,-0.453592,0.625592,1.401883,-1.285883,-0.509592,0.569592,1.345883
2,213,0.8,0.5,21,15,8,2.2,0.22,2.6,0.15,...,0.555143,1.234398,-2.283795,-0.925286,0.963286,2.321795,-1.957824,-0.793388,0.825388,1.989824
3,654,0.7,0.5,14,15,2,2.2,0.12,3.8,0.27,...,1.061837,2.323310,-1.205883,-0.429592,0.649592,1.425883,-2.790251,-1.140633,1.152633,2.802251
4,672,0.7,0.5,24,10,5,3.7,0.20,3.7,0.18,...,0.765041,1.638368,-2.977222,-1.133531,1.429531,3.273222,-2.100310,-0.838837,0.914837,2.176310


In [7]:
X_train = df.drop(['kfold', 'cent_price_cor', 'cent_trans_cor'], axis=1)
y_train = df[['cent_price_cor', 'cent_trans_cor']]

reg.fit(X_train, y_train)

y_pred = reg.predict(test_df.values)

df_submission = pd.DataFrame(y_pred, columns=['catboost_pred_price', 'catboost_pred_trans'])
df_submission

Learning rate set to 0.060689
0:	learn: 0.1165688	total: 8.86ms	remaining: 8.85s
1:	learn: 0.1165261	total: 15.5ms	remaining: 7.72s
2:	learn: 0.1164919	total: 21.4ms	remaining: 7.1s
3:	learn: 0.1164398	total: 27.6ms	remaining: 6.87s
4:	learn: 0.1163893	total: 33.8ms	remaining: 6.73s
5:	learn: 0.1163292	total: 40.2ms	remaining: 6.67s
6:	learn: 0.1162912	total: 46.4ms	remaining: 6.58s
7:	learn: 0.1162425	total: 52.4ms	remaining: 6.5s
8:	learn: 0.1161934	total: 58.3ms	remaining: 6.42s
9:	learn: 0.1161603	total: 64.2ms	remaining: 6.36s
10:	learn: 0.1161077	total: 70.2ms	remaining: 6.31s
11:	learn: 0.1160732	total: 76ms	remaining: 6.26s
12:	learn: 0.1160282	total: 82.3ms	remaining: 6.25s
13:	learn: 0.1159925	total: 88.5ms	remaining: 6.23s
14:	learn: 0.1159424	total: 94.6ms	remaining: 6.21s
15:	learn: 0.1159111	total: 101ms	remaining: 6.2s
16:	learn: 0.1158905	total: 107ms	remaining: 6.17s
17:	learn: 0.1158581	total: 112ms	remaining: 6.12s
18:	learn: 0.1157910	total: 118ms	remaining: 6.1s
19

167:	learn: 0.1102830	total: 986ms	remaining: 4.88s
168:	learn: 0.1102533	total: 992ms	remaining: 4.88s
169:	learn: 0.1102250	total: 998ms	remaining: 4.87s
170:	learn: 0.1101810	total: 1s	remaining: 4.86s
171:	learn: 0.1101480	total: 1.01s	remaining: 4.85s
172:	learn: 0.1101138	total: 1.01s	remaining: 4.84s
173:	learn: 0.1100683	total: 1.02s	remaining: 4.84s
174:	learn: 0.1100412	total: 1.02s	remaining: 4.83s
175:	learn: 0.1100030	total: 1.03s	remaining: 4.82s
176:	learn: 0.1099656	total: 1.03s	remaining: 4.81s
177:	learn: 0.1099262	total: 1.04s	remaining: 4.8s
178:	learn: 0.1098885	total: 1.05s	remaining: 4.8s
179:	learn: 0.1098346	total: 1.05s	remaining: 4.8s
180:	learn: 0.1097917	total: 1.06s	remaining: 4.8s
181:	learn: 0.1097413	total: 1.06s	remaining: 4.79s
182:	learn: 0.1097061	total: 1.07s	remaining: 4.79s
183:	learn: 0.1096563	total: 1.08s	remaining: 4.78s
184:	learn: 0.1096201	total: 1.08s	remaining: 4.77s
185:	learn: 0.1095798	total: 1.09s	remaining: 4.76s
186:	learn: 0.10954

337:	learn: 0.1033881	total: 1.99s	remaining: 3.89s
338:	learn: 0.1033455	total: 1.99s	remaining: 3.89s
339:	learn: 0.1033083	total: 2s	remaining: 3.88s
340:	learn: 0.1032659	total: 2s	remaining: 3.88s
341:	learn: 0.1032208	total: 2.01s	remaining: 3.87s
342:	learn: 0.1031677	total: 2.02s	remaining: 3.86s
343:	learn: 0.1031297	total: 2.02s	remaining: 3.86s
344:	learn: 0.1030853	total: 2.03s	remaining: 3.85s
345:	learn: 0.1030365	total: 2.03s	remaining: 3.85s
346:	learn: 0.1030065	total: 2.04s	remaining: 3.84s
347:	learn: 0.1029581	total: 2.05s	remaining: 3.83s
348:	learn: 0.1029140	total: 2.05s	remaining: 3.83s
349:	learn: 0.1028657	total: 2.06s	remaining: 3.82s
350:	learn: 0.1028230	total: 2.06s	remaining: 3.82s
351:	learn: 0.1027877	total: 2.07s	remaining: 3.81s
352:	learn: 0.1027566	total: 2.08s	remaining: 3.81s
353:	learn: 0.1027043	total: 2.08s	remaining: 3.8s
354:	learn: 0.1026623	total: 2.09s	remaining: 3.79s
355:	learn: 0.1026254	total: 2.09s	remaining: 3.79s
356:	learn: 0.10257

504:	learn: 0.0972821	total: 2.98s	remaining: 2.92s
505:	learn: 0.0972404	total: 2.99s	remaining: 2.92s
506:	learn: 0.0972033	total: 3s	remaining: 2.92s
507:	learn: 0.0971694	total: 3s	remaining: 2.91s
508:	learn: 0.0971242	total: 3.01s	remaining: 2.9s
509:	learn: 0.0970998	total: 3.02s	remaining: 2.9s
510:	learn: 0.0970772	total: 3.02s	remaining: 2.89s
511:	learn: 0.0970350	total: 3.03s	remaining: 2.89s
512:	learn: 0.0970058	total: 3.04s	remaining: 2.88s
513:	learn: 0.0969769	total: 3.04s	remaining: 2.88s
514:	learn: 0.0969505	total: 3.05s	remaining: 2.87s
515:	learn: 0.0969094	total: 3.05s	remaining: 2.86s
516:	learn: 0.0968713	total: 3.06s	remaining: 2.86s
517:	learn: 0.0968355	total: 3.06s	remaining: 2.85s
518:	learn: 0.0967962	total: 3.07s	remaining: 2.85s
519:	learn: 0.0967712	total: 3.08s	remaining: 2.84s
520:	learn: 0.0967350	total: 3.08s	remaining: 2.84s
521:	learn: 0.0967063	total: 3.09s	remaining: 2.83s
522:	learn: 0.0966731	total: 3.1s	remaining: 2.82s
523:	learn: 0.0966494

673:	learn: 0.0916287	total: 3.97s	remaining: 1.92s
674:	learn: 0.0915914	total: 3.97s	remaining: 1.91s
675:	learn: 0.0915536	total: 3.98s	remaining: 1.91s
676:	learn: 0.0915178	total: 3.99s	remaining: 1.9s
677:	learn: 0.0914834	total: 3.99s	remaining: 1.9s
678:	learn: 0.0914550	total: 4s	remaining: 1.89s
679:	learn: 0.0914252	total: 4s	remaining: 1.88s
680:	learn: 0.0914010	total: 4.01s	remaining: 1.88s
681:	learn: 0.0913808	total: 4.01s	remaining: 1.87s
682:	learn: 0.0913516	total: 4.02s	remaining: 1.87s
683:	learn: 0.0913182	total: 4.03s	remaining: 1.86s
684:	learn: 0.0912967	total: 4.03s	remaining: 1.85s
685:	learn: 0.0912701	total: 4.04s	remaining: 1.85s
686:	learn: 0.0912402	total: 4.04s	remaining: 1.84s
687:	learn: 0.0912086	total: 4.05s	remaining: 1.84s
688:	learn: 0.0911610	total: 4.05s	remaining: 1.83s
689:	learn: 0.0911362	total: 4.06s	remaining: 1.82s
690:	learn: 0.0911044	total: 4.07s	remaining: 1.82s
691:	learn: 0.0910801	total: 4.07s	remaining: 1.81s
692:	learn: 0.091050

...

347:	learn: 0.1015974	total: 2.46s	remaining: 4.62s
348:	learn: 0.1015612	total: 2.47s	remaining: 4.61s
349:	learn: 0.1015276	total: 2.48s	remaining: 4.61s
350:	learn: 0.1014942	total: 2.49s	remaining: 4.6s
351:	learn: 0.1014733	total: 2.49s	remaining: 4.59s
352:	learn: 0.1014359	total: 2.5s	remaining: 4.58s
353:	learn: 0.1014016	total: 2.51s	remaining: 4.58s
354:	learn: 0.1013698	total: 2.51s	remaining: 4.57s
355:	learn: 0.1013450	total: 2.52s	remaining: 4.56s
356:	learn: 0.1013021	total: 2.53s	remaining: 4.55s
357:	learn: 0.1012638	total: 2.53s	remaining: 4.54s
358:	learn: 0.1012157	total: 2.54s	remaining: 4.54s
359:	learn: 0.1011649	total: 2.55s	remaining: 4.53s
360:	learn: 0.1011159	total: 2.55s	remaining: 4.52s
361:	learn: 0.1010872	total: 2.56s	remaining: 4.51s
362:	learn: 0.1010442	total: 2.57s	remaining: 4.5s
363:	learn: 0.1010167	total: 2.57s	remaining: 4.5s
364:	learn: 0.1009768	total: 2.58s	remaining: 4.49s
365:	learn: 0.1009447	total: 2.58s	remaining: 4.48s
366:	learn: 0.10

523:	learn: 0.0951843	total: 3.67s	remaining: 3.33s
524:	learn: 0.0951408	total: 3.67s	remaining: 3.32s
525:	learn: 0.0950998	total: 3.68s	remaining: 3.32s
526:	learn: 0.0950609	total: 3.69s	remaining: 3.31s
527:	learn: 0.0950255	total: 3.69s	remaining: 3.3s
528:	learn: 0.0949831	total: 3.7s	remaining: 3.3s
529:	learn: 0.0949469	total: 3.71s	remaining: 3.29s
530:	learn: 0.0949181	total: 3.72s	remaining: 3.28s
531:	learn: 0.0948823	total: 3.72s	remaining: 3.27s
532:	learn: 0.0948552	total: 3.73s	remaining: 3.27s
533:	learn: 0.0948145	total: 3.74s	remaining: 3.26s
534:	learn: 0.0947745	total: 3.74s	remaining: 3.25s
535:	learn: 0.0947414	total: 3.75s	remaining: 3.25s
536:	learn: 0.0947179	total: 3.76s	remaining: 3.24s
537:	learn: 0.0946807	total: 3.77s	remaining: 3.23s
538:	learn: 0.0946466	total: 3.77s	remaining: 3.23s
539:	learn: 0.0946091	total: 3.78s	remaining: 3.22s
540:	learn: 0.0945819	total: 3.79s	remaining: 3.21s
541:	learn: 0.0945378	total: 3.8s	remaining: 3.21s
542:	learn: 0.09

693:	learn: 0.0894480	total: 4.88s	remaining: 2.15s
694:	learn: 0.0894227	total: 4.89s	remaining: 2.15s
695:	learn: 0.0894025	total: 4.9s	remaining: 2.14s
696:	learn: 0.0893662	total: 4.9s	remaining: 2.13s
697:	learn: 0.0893393	total: 4.91s	remaining: 2.13s
698:	learn: 0.0893213	total: 4.92s	remaining: 2.12s
699:	learn: 0.0892835	total: 4.92s	remaining: 2.11s
700:	learn: 0.0892486	total: 4.93s	remaining: 2.1s
701:	learn: 0.0892173	total: 4.94s	remaining: 2.1s
702:	learn: 0.0891844	total: 4.95s	remaining: 2.09s
703:	learn: 0.0891557	total: 4.95s	remaining: 2.08s
704:	learn: 0.0891195	total: 4.96s	remaining: 2.08s
705:	learn: 0.0890992	total: 4.97s	remaining: 2.07s
706:	learn: 0.0890620	total: 4.97s	remaining: 2.06s
707:	learn: 0.0890328	total: 4.98s	remaining: 2.05s
708:	learn: 0.0890018	total: 4.99s	remaining: 2.05s
709:	learn: 0.0889684	total: 4.99s	remaining: 2.04s
710:	learn: 0.0889394	total: 5s	remaining: 2.03s
711:	learn: 0.0889197	total: 5s	remaining: 2.02s
712:	learn: 0.0888845	

873:	learn: 0.0840509	total: 6.07s	remaining: 875ms
874:	learn: 0.0840318	total: 6.08s	remaining: 868ms
875:	learn: 0.0840103	total: 6.08s	remaining: 861ms
876:	learn: 0.0839794	total: 6.09s	remaining: 854ms
877:	learn: 0.0839544	total: 6.1s	remaining: 847ms
878:	learn: 0.0839280	total: 6.11s	remaining: 840ms
879:	learn: 0.0838822	total: 6.11s	remaining: 833ms
880:	learn: 0.0838615	total: 6.12s	remaining: 827ms
881:	learn: 0.0838337	total: 6.13s	remaining: 820ms
882:	learn: 0.0838000	total: 6.14s	remaining: 813ms
883:	learn: 0.0837798	total: 6.15s	remaining: 807ms
884:	learn: 0.0837526	total: 6.15s	remaining: 800ms
885:	learn: 0.0837275	total: 6.16s	remaining: 793ms
886:	learn: 0.0836953	total: 6.17s	remaining: 786ms
887:	learn: 0.0836600	total: 6.18s	remaining: 779ms
888:	learn: 0.0836359	total: 6.19s	remaining: 773ms
889:	learn: 0.0836130	total: 6.2s	remaining: 766ms
890:	learn: 0.0835914	total: 6.21s	remaining: 759ms
891:	learn: 0.0835678	total: 6.21s	remaining: 752ms
892:	learn: 0.

,catboost_pred_price,catboost_pred_trans
0,-0.172750,0.323119
1,-0.213441,0.407223
2,-0.201747,0.354451
3,-0.143301,0.350456
4,-0.196469,0.348525
...,...,...
8054,-0.216853,0.381874
8055,-0.215520,0.370741
8056,-0.203969,0.343413
8057,-0.188443,0.373622


In [8]:
df_submission.to_csv('model_preds/catboost_sub.csv', index=False)